# Bloque 2 · Conectando RAG con la nube

En este bloque daremos el salto de la simulación local a un servicio en la nube. Presentaremos conceptos básicos de computación en la nube, APIs y Amazon Bedrock, para luego analizar cómo luce la respuesta del endpoint `Retrieve` cuando contamos con una knowledge base ya configurada.

## De la simulación a la nube

- En el **Bloque 0** visualizamos la arquitectura completa y los objetivos del proyecto.
- En el **Bloque 1** exploramos la recuperación basada en similitud con una simulación local.

Ahora nos enfocamos en cómo un servicio administrado nos ayuda a escalar esa lógica, manteniendo el hilo conductor: recuperar fragmentos relevantes para enriquecer respuestas generadas.

## Conceptos clave: nube y APIs

- **Computación en la nube**: consumo de recursos (cómputo, almacenamiento, modelos) como servicios bajo demanda, sin gestionar infraestructura física.
- **API (Application Programming Interface)**: conjunto de reglas para que dos sistemas se comuniquen. Usualmente exponemos recursos vía URLs (endpoints) que aceptan solicitudes con parámetros y devuelven respuestas estructuradas (JSON).
- **Endpoint**: dirección específica de una API que realiza una acción. Por ejemplo, `Retrieve` es un endpoint que consulta una base de conocimiento con una pregunta.

## ¿Qué ofrece Amazon Bedrock?

Amazon Bedrock es un servicio administrado que permite acceder a modelos fundacionales (propios y de terceros) mediante una API unificada. Para nuestro flujo RAG, destacamos:
- **Modelos de embeddings** para convertir texto en vectores.
- **Modelos generativos** para elaborar respuestas.
- **Knowledge bases** que almacenan representaciones vectoriales y resuelven la recuperación de forma administrada.
- **Integración con otras herramientas AWS** (S3, IAM, CloudWatch) que facilitan la operación en producción.

## Knowledge Bases en Bedrock

Una *knowledge base* es un repositorio vectorial que Bedrock administra por nosotros. Sus componentes principales son:
- **Fuentes de datos**: conjuntos de documentos almacenados, por ejemplo, en Amazon S3. Podemos proveer manuales, políticas, FAQs, etc.
- **Pipelines de ingesta**: procesos que limpian, fragmentan y generan embeddings de cada documento.
- **Catálogo de metadatos**: guarda referencias al origen de cada fragmento (ruta de archivo, sección, versión) para poder citar las fuentes en las respuestas.

Asumimos que la organización ya configuró la knowledge base y ejecutó al menos una ingesta para tener información disponible.

## Repaso rápido: embeddings

Los *embeddings* convierten texto en vectores de números que preservan el significado. Fragmentos que hablan de lo mismo quedan cerca entre sí en el espacio vectorial. En un flujo RAG:
- Tomamos la pregunta del usuario y generamos su embedding.
- Buscamos los vectores más cercanos dentro de la knowledge base.
- Recuperamos los fragmentos asociados para proveer contexto al modelo generativo.

En Amazon Bedrock podemos elegir distintos modelos de embeddings; cada uno ofrece diferentes costos, dimensiones y optimizaciones (por ejemplo, Titan Text Embeddings v2).

## ¿Qué es boto3 y cómo se autentica?

`boto3` es el SDK oficial de AWS para Python. Nos permite crear clientes para cada servicio y llamar a sus APIs con objetos Python. La autenticación suele resolverse de tres formas principales:
- Variables de entorno (`AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_SESSION_TOKEN`, `AWS_REGION`).
- Archivos de credenciales en `~/.aws/credentials` administrados por AWS CLI o SSO.
- Roles asumidos automáticamente cuando ejecutamos el código en un servicio administrado de AWS (por ejemplo, Lambda o SageMaker).

En este bloque crearemos un cliente de `bedrock-agent-runtime` usando boto3 para invocar el endpoint `Retrieve` de una knowledge base existente.

## Endpoint `Retrieve`

- Recibe una **pregunta** o consulta en texto mediante la API `bedrock-agent-runtime`.
- Genera el embedding de la consulta con el modelo configurado en la knowledge base.
- Ejecuta una búsqueda vectorial (puedes controlar parámetros como `numberOfResults`).
- Devuelve un listado ordenado de fragmentos con puntuaciones, contenido y metadatos.

En Python accederemos a este endpoint usando `boto3.client("bedrock-agent-runtime").retrieve(...)`. La respuesta será un diccionario JSON que luego transformaremos en un formato amigable para mostrar al usuario o alimentar la fase generativa.

## Actividad práctica · Hacer una pregunta a la knowledge base

Invocaremos el endpoint `Retrieve` con boto3 para obtener fragmentos reales de una knowledge base ya configurada. La meta es comprender qué campos devuelve la API y cómo parsearlos en Python para alimentar la siguiente etapa del flujo RAG.

### Preparación antes de ejecutar

1. Asegúrate de tener credenciales de AWS válidas en tu entorno (variables de entorno, AWS CLI, SSO, etc.).
2. Identifica el `knowledgeBaseId` de la knowledge base que quieres consultar y asígnalo a la variable de entorno `BEDROCK_KB_ID`.
3. Opcional: define `AWS_REGION` si tu knowledge base está en una región distinta a `us-east-1`.
4. Ejecuta la celda de instalación de dependencias en el **Bloque 0** (o instala manualmente los paquetes del `requirements.txt`).

Con esto listo, estaremos preparados para invocar el endpoint `Retrieve` directamente desde la notebook.

In [ ]:
import os
from typing import Any, Dict, Optional

import boto3

AWS_REGION = os.environ.get("AWS_REGION", "us-east-1")
KNOWLEDGE_BASE_ID = os.environ.get("BEDROCK_KB_ID")


def realizar_consulta(
    pregunta: str,
    top_k: int = 3,
    kb_id: Optional[str] = None,
    region: Optional[str] = None,
) -> Dict[str, Any]:
    """Invoca el endpoint Retrieve de Amazon Bedrock para una knowledge base dada."""
    kb_identificador = kb_id or KNOWLEDGE_BASE_ID
    if not kb_identificador:
        raise ValueError(
            "Define la variable de entorno BEDROCK_KB_ID con el identificador de tu knowledge base."
        )

    cliente = boto3.client(
        "bedrock-agent-runtime",
        region_name=region or AWS_REGION,
    )

    respuesta = cliente.retrieve(
        knowledgeBaseId=kb_identificador,
        retrievalConfiguration={
            "vectorSearchConfiguration": {
                "numberOfResults": top_k,
            }
        },
        input={"text": pregunta},
    )
    return respuesta

Observa que la respuesta incluye:
- El identificador de la knowledge base utilizada (`knowledgeBaseId`).
- El ARN del modelo de embeddings (`modelArn`).
- La pregunta original (`query`).
- Un arreglo de `retrievalResults` con puntajes (`score`), contenido y metadatos.
- Información de la ubicación del documento para citar la fuente.

En los próximos pasos enviaremos una pregunta, capturaremos la respuesta y la transformaremos en un formato más legible para nuestro chatbot.

In [ ]:
import textwrap


def mostrar_resultados(respuesta: Dict[str, Any], max_ancho: int = 96) -> None:
    for idx, resultado in enumerate(respuesta.get("retrievalResults", []), start=1):
        score = round(resultado.get("score", 0.0) * 100, 2)
        metadatos = resultado.get("metadata", {})
        texto = resultado.get("content", [{}])[0].get("text", "")

        print(f"Resultado #{idx}")
        print(f"Similitud: {score}%")
        print(f"Título: {metadatos.get('title', 'Sin título')}")
        print(f"Fuente: {metadatos.get('source', 'Sin referencia')}")
        print("Fragmento:")
        for linea in textwrap.wrap(texto, width=max_ancho):
            print(f"  {linea}")
        print("-" * max_ancho)


pregunta_demo = "¿Cómo prepara Bedrock una knowledge base para responder preguntas?"

respuesta_obtenida = realizar_consulta(pregunta_demo, top_k=3)
mostrar_resultados(respuesta_obtenida)


### Experimenta

- Configura `BEDROCK_KB_ID` (y, si corresponde, `AWS_REGION`) para apuntar a tu knowledge base real.
- Cambia `pregunta_demo` y observa cómo varían los fragmentos recuperados.
- Ajusta `top_k` en la llamada a `realizar_consulta` para traer más o menos contexto.
- Extiende `mostrar_resultados` para incluir citas, enlaces o cualquier dato que tu chatbot necesite mostrar al usuario.
- Integra los fragmentos en tu prompt de generación para practicar cómo construir respuestas explicables.

## Próximos pasos

En el siguiente bloque utilizaremos la API de RAG (`retrieve_and_generate`) de Amazon Bedrock para combinar la recuperación con un modelo generativo y construiremos la interacción en Chainlit. Aprovecharemos los fragmentos y metadatos parseados aquí para elaborar respuestas completas y trazables.